In [127]:
import cv2
import numpy as np
import dlib
import time
import pyautogui

In [128]:
## 인식기 load 및 상수 파라미터 지정
# 얼굴 인식 model, 랜드마크 인식기 생성
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./content/shape_predictor_68_face_landmarks.dat')
pyautogui.FAILSAFE = False

# 양 눈 영역을 나타내는 랜드마크 인덱스
left_eye_landmarks  = [36, 37, 38, 39, 40, 41]
right_eye_landmarks  = [42, 43, 44, 45, 46, 47]

output_width, output_height = 1920, 1080 # 내 모니터 출력창 크기
output_ratio = 1

In [129]:
## 초기값 설정
THRESHOLD_BIN = 50 # 이진화 임계값 설정 (: 클수록 화소값=0인 영역(검정색영역) 넓어짐)
#global threshold_dis # 좌표 차이 임계값 (: 작을수록 시선움직임에 민감하게 반응)
THRESHOLD_DIS = 9
#global sensitivity # 민감도 (: 클수록 마우스의 움직임이 커진다)
SENSITIVITY = 80
dx = dy = 0
#pyautogui.doubleClickInterval = 1.0
global x_prev, y_prev
x_prev = y_prev = None
# 눈의 감김 비율 상한
EYE_AR_THRESH = 0.35
# 감김 상태 타이머 초기값
TIMER_THRESH = 1
# 눈의 감김 상태를 저장하는 변수
left_eye_closed = False
right_eye_closed = False
# 눈의 감김 상태를 저장하는 타이머
left_eye_timer = 0
right_eye_timer = 0

In [130]:
def eye_aspect_ratio(eye_np): # 눈의 감김 여부를 확인하는 함수
    if eye_np is not None: # 눈의 수평 방향 좌표 거리 계산
        A = abs(eye_np[1][1] - eye_np[5][1])
        B = abs(eye_np[2][1] - eye_np[4][1])
        C = abs(eye_np[3][0] - eye_np[0][0])

        ear = (A + B) / (2.0 * C)
        ear = round(ear, 2)
        return ear
    else:
        return 404

In [131]:
def resize_and_show(name, img):
    if img is not None:
        _h, _w = img.shape[:2]
        cv2.namedWindow(name, cv2.WINDOW_NORMAL)  
        cv2.resizeWindow(name, _w*output_ratio, _h*output_ratio) 
        if name == "L": 
            cv2.moveWindow(name, int(output_width/2)-(320+_w*output_ratio), int(output_height* 0.05))
        else:
            cv2.moveWindow(name, int(output_width/2)+(130+_w*output_ratio), int(output_height* 0.05))
        cv2.imshow(name, img)

In [132]:
def crop_and_binarization(img, rect, name):
    # 이미지 자르기
    x1, y1 = rect[0]
    x2, y2 = rect[2]
    cropped_img = img[y1:y2, x1:x2]

    # 자른 이미지를 이진화
    #binary_image = get_binary_img(cropped_img)
    gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    blur = cv2.GaussianBlur(equalized, (5, 5), 0)
    _, binary_image = cv2.threshold(blur, THRESHOLD_BIN, 255, cv2.THRESH_BINARY)

    # 객체 영역과 나머지 영역을 분할하여 결과 이미지 생성
    binarized_img = np.where(binary_image <= THRESHOLD_BIN, 0, 255).astype(np.uint8)
    
    # resize and show
    resize_and_show(name, binarized_img)

    return binarized_img


In [133]:
def find_centroid(binary_image):
    # 검은색 영역의 좌표 찾기
    black_pixels = np.where(binary_image == 0)
    num_black_pixels = len(black_pixels[0])

    x_coords = black_pixels[1]
    y_coords = black_pixels[0]

    # 무게 중심을 계산하기 위해 좌표의 합과 픽셀 개수 계산
    sum_x = np.sum(x_coords)
    sum_y = np.sum(y_coords)
    num_pixels = num_black_pixels

    # 무게 중심 계산
    centroid_x = sum_x / num_pixels
    centroid_y = sum_y / num_pixels

    return centroid_x, centroid_y



In [134]:
def init_point():
    pyautogui.moveTo(output_width//2, output_height//2)
    time.sleep(1)

In [135]:
def get_eye_part(landmarks, eye):
    eye_part = np.empty((0, 2), dtype=float)
    for i in range(0,6):
        eye_part = np.append(eye_part, 
                             np.array([[landmarks.part(eye[i]).x, 
                                        landmarks.part(eye[i]).y]]), 
                                        axis=0)
    return eye_part

In [136]:
def get_eye_pts(landmarks, eye):
    # 눈 영역(사각형)의 x, y 좌표를 저장할 리스트 초기화
    eye_pts = []
    # 눈 영역을 crop할 사각 바운더리 계산
    mid_side_h = np.mean([landmarks.part(eye[0]).y, landmarks.part(eye[3]).y])
    up_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[1]).y, landmarks.part(eye[2]).y]))
    up_side = mid_side_h - up_side_dd
    low_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[4]).y, landmarks.part(eye[5]).y]))
    low_side = mid_side_h + low_side_dd
    left_side = landmarks.part(eye[0]).x 
    right_side = landmarks.part(eye[3]).x 

    eye_pts.append((left_side, up_side))
    eye_pts.append((right_side, up_side)) 
    eye_pts.append((right_side, low_side)) 
    eye_pts.append((left_side, low_side))     

    eye_pts_np = np.array(eye_pts, np.int32)

    return eye_pts_np

In [137]:
def draw_text(img, text, pos): # 출력 창에 text를 표시
    font = cv2.FONT_HERSHEY_SIMPLEX
    color=(0, 255, 0)
    thickness=2
    scale=0.7
    size, _ = cv2.getTextSize(text, font, scale, thickness)
    x, y = pos
    cv2.putText(img, text, (x - size[0]//2, y + size[1]//2), 
                font, scale, color, thickness, cv2.LINE_AA)

In [138]:
def process(point):
    global x_prev, y_prev
    state_s = False

    x, y = point # 좌표 추출

    if x_prev is not None and y_prev is not None:
        dx = x - x_prev
        dy = y - y_prev
        
        distance = dx**2 + dy**2 # 이동 거리 계산
        
        # 정지 모드 체크
        if distance < THRESHOLD_DIS:
            state_s = True

        x_prev = x  # 갱신
        y_prev = y
    else:
        x_prev = x
        y_prev = y
        dx = dy = 0
    
    return dx, dy, state_s


In [139]:
def show_68_landmarks(frame, landmarks):
    for point in landmarks:
        x, y = point[0], point[1]
        cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

In [140]:
## main()
# ##실행 시 반드시 얼굴이 있어야 오류가 뜨지 않음##
init_trig = True
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Failed to open webcam")

cv2.namedWindow("Result", cv2.WINDOW_NORMAL)
cv2.moveWindow("Result", int(output_width/2)-180, int(output_height* 0.05))

while True:
    # 프레임 읽어오기
    ret, frame = cap.read()
    # 프레임 크기 조정 (속도 향상을 위해)
    frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    frame = cv2.flip(frame, 1)
    frame_height = frame.shape[0] 
    frame_width = frame.shape[1] # 여러 정보를 출력창에 표시하기 위해 창 크기 파악

    # 영점보정 트리거=="i" 시 -> 마우스를 화면의 정중앙으로 이동(1초대기)
    if cv2.waitKey(1) & 0xFF == ord('i'):
        init_trig = True
    if init_trig:
        init_point()
        init_trig = False

    # 그레이스케일 후 얼굴 검출
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if not faces:
        continue
    for face in faces:
        # 얼굴의 68개 landmarks 검출 
        landmarks = predictor(gray, face)
        landmarks_show = [(p.x, p.y) for p in landmarks.parts()]
        parts = landmarks.parts()

        # 양 눈의 랜드마크 추출 (landmarks tuple -> np배열로 변환)
        left_eye_part = get_eye_part(landmarks, left_eye_landmarks)
        right_eye_part = get_eye_part(landmarks, right_eye_landmarks)

        # 양 눈의 감지 창(window)범위 계산
        left_eye_win = get_eye_pts(landmarks, left_eye_landmarks)
        right_eye_win = get_eye_pts(landmarks, right_eye_landmarks)

        # 눈 위치 기반으로 안구에 대한 이미지를 자르고 이진화 및 출력
        left_eye_biimg = crop_and_binarization(frame, left_eye_win, "L")
        right_eye_biimg = crop_and_binarization(frame, right_eye_win, "R")
        
        # eye 창 내에서 이진화 후 무게중심값 추출 > 시선 좌표 탐지
        left_center = find_centroid(left_eye_biimg) # None값 나오지 않음
        right_center = find_centroid(right_eye_biimg)

        show_68_landmarks(frame, landmarks_show)

        ## 눈 감음 -> 마우스 동작 은 한쪽 눈을 감은채로 시섬 움직임에 영향을 받지 x
        ## 시선 움직임 -> 마우스 이동 은 두쪽눈 모두 뜨고있는 경우에만 동작
        ## 더블클릭또한 두쪽눈 다 뜬채로 같은 곳을 3초 응시할 떄만 동작
        # 눈의 감김 여부 확인 (param: ndarray)
        left_eye_ear = eye_aspect_ratio(left_eye_part)
        right_eye_ear = eye_aspect_ratio(right_eye_part)

        # 눈의 감김 여부에 따라 상태 업데이트 (왼)
        if left_eye_ear < EYE_AR_THRESH: # 눈 감김이 감지된 경우,
            if left_eye_closed == True: # 계속 감고 있다면 3초되는 순간 좌클릭 1번
                left_eye_timer = time.time() - start_time_L
                text = "L_closed"
                draw_text(frame, text, (int(50), int(frame_height*0.95)))
                if left_eye_timer >= TIMER_THRESH:
                    pyautogui.click(button='left')
                    left_eye_closed = False
            else: 
                start_time_L = time.time() # 눈을 감은 직후라면 타이머 시작
                left_eye_closed = True
        else: # 그 외: 도중에 눈 뜨거나 그냥 눈 뜨고있는 중이면 타이머 초기화
            left_eye_timer = 0
            left_eye_closed = False

        # 눈의 감김 여부에 따라 상태 업데이트 (오)
        if right_eye_ear < EYE_AR_THRESH:
            if right_eye_closed == True: # 계속 감고 있다면 3초가되는 순간 우클릭 1번
                right_eye_timer = time.time() - start_time_R
                text = "R_closed"
                draw_text(frame, text, (int(frame_width-50), int(frame_height*0.95)))
                if right_eye_timer >= TIMER_THRESH:
                    pyautogui.click(button='right')
                    right_eye_closed = False
            else:
                start_time_R = time.time() # 눈을 감은 직후라면 타이머 시작
                right_eye_closed = True
        else:
            right_eye_timer = 0
            right_eye_closed = False

        # 두 눈을 모두 뜨고 있을 경우
        # 1. 몇 초이상 같은 곳을 응시할 경우(정지상태) 더블클릭
        # 2. 움직일 경우 마우스 이동
        if left_eye_ear >= EYE_AR_THRESH and right_eye_ear >= EYE_AR_THRESH:
            average_x = (left_center[0] + right_center[0]) / 2
            average_y = (left_center[1] + right_center[1]) / 2
            point = (average_x, average_y)
            dx, dy, state_s = process(point)

            if state_s == True: # 1.TIMER_THRESH초이상 같은 곳을 응시할 경우(정지상태) 더블클릭
                if timer == True:
                    elapsed_time = time.time() - start_time
                    if elapsed_time >= TIMER_THRESH:
                        pyautogui.doubleClick()
                        timer = False
                else:
                    start_time = time.time()
                    timer = True
            else: # 2. 움직일 경우 마우스 이동
                pyautogui.move(dx*SENSITIVITY, dy*SENSITIVITY)
                timer = False

        text = f"({round(dx, 2)}, {round(dy, 2)})"  # 출력할 좌표값 문자열
        draw_text(frame, text, (int(frame_width/2), int(frame_height*0.95)))  # 중앙 하단에 출력


    # 화면에 프레임 출력
    cv2.imshow("Result", frame)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠 해제 및 창 닫기
cap.release()
cv2.destroyAllWindows()